**Train Test and Cross Validation**

In [2]:
import pandas as pd
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score


In [3]:
# Load training data
data_path = '/workspaces/Final-Year-Project/Cleaned Data/TrainTestData.csv'
df = pd.read_csv(data_path)

# Display the original shape of the dataset
print(f"🔍 AB - Original dataset shape: {df.shape}")

# Drop missing values
df = df.dropna()

# Display the shape after dropping NaNs
print(f"🧹 AB - After dropping NaNs: {df.shape}")

# Features and target variable
X = df.drop('Diabetes Status', axis=1)
y = df['Diabetes Status']


🔍 AB - Original dataset shape: (3268, 14)
🧹 AB - After dropping NaNs: (3268, 14)


In [4]:
# Feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Display the shape after scaling
print(f"📊 AB - Final dataset shape after scaling: {X_scaled.shape}")


📊 AB - Final dataset shape after scaling: (3268, 13)


In [5]:
# AdaBoost Classifier setup
ab = AdaBoostClassifier(random_state=42)

# Hyperparameter grid for GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 150, 200],  # Number of base models (trees) in the ensemble
    'learning_rate': [0.01, 0.05, 0.1, 0.2],  # Impact of each base model
}

# KFold cross-validation setup
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# GridSearchCV setup with F1-score as the scoring parameter
grid_search = GridSearchCV(ab, param_grid, scoring='f1', cv=kf, n_jobs=-1, verbose=1)


In [6]:
# Fit GridSearchCV on the training data
grid_search.fit(X_scaled, y)

# Best model from GridSearchCV
best_ab = grid_search.best_estimator_

# Print the best hyperparameters and the best score
print(f"✅ AB - Best Parameters: {grid_search.best_params_}")
print(f"✅ AB - K-Fold Mean F1-Score with Best Parameters: {grid_search.best_score_:.4f}")


Fitting 5 folds for each of 16 candidates, totalling 80 fits


✅ AB - Best Parameters: {'learning_rate': 0.2, 'n_estimators': 200}
✅ AB - K-Fold Mean F1-Score with Best Parameters: 0.7674


In [7]:
# Predictions on training data
y_pred = best_ab.predict(X_scaled)

# Classification report on the training data
print("📌 AB Classification Report on Training Data:")
print(classification_report(y, y_pred))

# Confusion matrix on the training data
print("🟦 AB - Confusion Matrix on Training Data:")
print(confusion_matrix(y, y_pred))


📌 AB Classification Report on Training Data:
              precision    recall  f1-score   support

           0       0.82      0.64      0.72      1638
           1       0.70      0.86      0.77      1630

    accuracy                           0.75      3268
   macro avg       0.76      0.75      0.74      3268
weighted avg       0.76      0.75      0.74      3268

🟦 AB - Confusion Matrix on Training Data:
[[1041  597]
 [ 227 1403]]


In [8]:
# Load validation data
valid_path = "/workspaces/Final-Year-Project/Cleaned Data/ValidationData.csv"
df_valid = pd.read_csv(valid_path)

# Drop any rows with missing values in the validation set
df_valid = df_valid.dropna()

# Prepare features and target for validation data
X_valid = df_valid.drop('Diabetes Status', axis=1)
y_valid = df_valid['Diabetes Status']

# Scale the validation data using the previously fitted scaler
X_valid_scaled = scaler.transform(X_valid)

# Display the shape of the validation dataset
print(f"🧪 AB - Validation dataset shape: {X_valid.shape}")


🧪 AB - Validation dataset shape: (364, 13)


In [9]:
# Predictions on validation data
y_pred_valid = best_ab.predict(X_valid_scaled)


In [10]:
# Print classification report for validation data
print(f"📌 AB - Validation Classification Report:")
print(classification_report(y_valid, y_pred_valid))

# Print confusion matrix for validation data
print("🟦 AB - Validation Confusion Matrix:")
print(confusion_matrix(y_valid, y_pred_valid))


📌 AB - Validation Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.62      0.71       178
           1       0.71      0.88      0.78       186

    accuracy                           0.75       364
   macro avg       0.77      0.75      0.75       364
weighted avg       0.77      0.75      0.75       364

🟦 AB - Validation Confusion Matrix:
[[111  67]
 [ 23 163]]


In [11]:
import pandas as pd
from sklearn.metrics import classification_report

# Assuming you already have y_valid (true labels) and y_pred_valid (predicted labels)
# Example classification report for AdaBoost (AB)
report = classification_report(y_valid, y_pred_valid, output_dict=True)

# Extract weighted scores
weighted_precision = report['weighted avg']['precision']
weighted_recall = report['weighted avg']['recall']
weighted_f1 = report['weighted avg']['f1-score']
accuracy = report['accuracy']

# Create a dictionary with the scores
ab_results = {
    'Model': 'AdaBoost (AB)',
    'Precision': weighted_precision,
    'Recall': weighted_recall,
    'F1-Score': weighted_f1,
    'Accuracy': accuracy
}

# Convert the dictionary to a DataFrame
ab_df = pd.DataFrame([ab_results])

# Create a DataFrame to store results (if not already created)
results_df = pd.DataFrame(columns=['Model', 'Precision', 'Recall', 'F1-Score', 'Accuracy'])

# Concatenate the new row to the DataFrame
results_df = pd.concat([results_df, ab_df], ignore_index=True)

# Print the DataFrame to confirm it's added
print(results_df)

# Save the DataFrame to a CSV file
results_df.to_csv("/workspaces/Final-Year-Project/Results/model_results.csv", index=False)


           Model  Precision    Recall  F1-Score  Accuracy
0  AdaBoost (AB)   0.767212  0.752747  0.748389  0.752747


/tmp/ipykernel_8804/645382371.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, ab_df], ignore_index=True)


In [12]:
# Recalculate Macro average F1-score on validation data
validation_f1_macro = f1_score(y_valid, y_pred_valid, average='macro')
print(f"🏆 Best Model Validation F1-Score: {validation_f1_macro}")


🏆 Best Model Validation F1-Score: 0.7475961538461539


In [13]:
# Assuming you already have the correct training data variables: X_train, y_train, and y_train_pred
# Train results
train_report = classification_report(y, y_pred, output_dict=True)  # Use the correct training predictions
train_results = {
    'Model': 'AdaBoost',
    'Precision (Train)': train_report['weighted avg']['precision'],
    'Recall (Train)': train_report['weighted avg']['recall'],
    'F1-Score (Train)': train_report['weighted avg']['f1-score'],
    'Accuracy (Train)': train_report['accuracy']
}

# Validation results
y_valid_pred = best_ab.predict(X_valid_scaled)  # Assuming you've already processed X_valid_scaled
valid_report = classification_report(y_valid, y_valid_pred, output_dict=True)
valid_results = {
    'Model': 'AdaBoost',
    'Precision (Validation)': valid_report['weighted avg']['precision'],
    'Recall (Validation)': valid_report['weighted avg']['recall'],
    'F1-Score (Validation)': valid_report['weighted avg']['f1-score'],
    'Accuracy (Validation)': valid_report['accuracy']
}

# Convert both to DataFrame and concatenate
train_df = pd.DataFrame([train_results])
valid_df = pd.DataFrame([valid_results])

# Combine the results into one DataFrame
results_df = pd.concat([train_df, valid_df], ignore_index=True)

# Display the combined DataFrame
print(results_df)

# Optionally, save the results to CSV
results_df.to_csv("ada_boost_results.csv", index=False)


      Model  Precision (Train)  Recall (Train)  F1-Score (Train)  \
0  AdaBoost           0.761385        0.747858          0.744657   
1  AdaBoost                NaN             NaN               NaN   

   Accuracy (Train)  Precision (Validation)  Recall (Validation)  \
0          0.747858                     NaN                  NaN   
1               NaN                0.767212             0.752747   

   F1-Score (Validation)  Accuracy (Validation)  
0                    NaN                    NaN  
1               0.748389               0.752747  
